<a href="https://colab.research.google.com/github/sirakik/japan_graph/blob/master/%E5%A4%A9%E5%9C%B0%E5%89%B5%E9%80%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#世界グラフを作る

SEKAI_DATA.ipynbでworld.csvを作成済み

In [1]:
!wget -q https://raw.githubusercontent.com/sirakik/japan_graph/master/the_creation/world.csv -O world.csv